In [1]:
import ee

In [3]:
ee.Authenticate()

Enter verification code:  4/1AX4XfWhe67PPKom7nH9xPd3H8qi-lrcEuzoquiqwiLFwfTAumRQJk6Qw7O0



Successfully saved authorization token.


In [2]:
ee.Initialize()

In [30]:
import importlib
from flood_detection import modis
importlib.reload(modis)
from flood_detection.utils import misc
import geemap
from pathlib import Path
import rasterio
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [80]:
districts = ee.FeatureCollection("projects/ee-nip/assets/ShapeFiles/Admin/Districts")

In [5]:
evi = ee.ImageCollection("MODIS/061/MOD13Q1")

In [6]:
geom = districts.union().geometry()

In [42]:
flood_map = modis.dfo(geom, str(year)+"-06-01", str(year)+"-12-31", "standard", '3Day', True)

flood_map = misc.apply_slope_mask(flood_map, thresh=5)

# perm_water = misc.get_jrc_perm(geom)

dfo_final = ee.Image(flood_map).select('flooded')

In [47]:
reducedRegions = dfo_final.reduceRegions(districts, ee.Reducer.mean(), 250, 'EPSG:4326')

In [53]:
data = reducedRegions.map(lambda feature: feature.setGeometry(None)).select(['NAME_2','mean']).getInfo()

In [58]:
dataList = [item['properties'] for item in data['features']]

In [71]:
df = pd.DataFrame(dataList)
df.columns = ['District', 'FractionFlooded']
df['Year'] = year

In [9]:
dfs = []
for year in range(2001, 2022):
    print(year)
    startDate = str(year)+"-06-01"
    endDate = str(year)+"-12-31"
    flood_map = modis.dfo(geom, startDate, endDate, "standard", '3Day', True)
    
    flood_map = misc.apply_slope_mask(flood_map, thresh=5)
    
    mask = ee.Image("projects/ee-gates/assets/Dataset/Raster/Inferred/RFRiceClassified/"+str(year)).eq(1)
    flood_map = flood_map.updateMask(mask)
    
    flood_map = flood_map.addBands(evi.filterDate(ee.Date(startDate), ee.Date(endDate)).select('EVI').median())

    dfo_final = ee.Image(flood_map).select(['flooded', 'duration' , 'max_img', 'EVI'])
    
    reducedRegions = dfo_final.reduceRegions(districts, ee.Reducer.mean(), 250, 'EPSG:4326')
    
    data = reducedRegions.map(lambda feature: feature.setGeometry(None)).select(['NAME_2','flooded', 'duration', 'max_img', 'EVI']).getInfo()
    dataList = [item['properties'] for item in data['features']]
    df = pd.DataFrame(dataList)
    df.columns = ['EVI', 'District', 'Duration', 'FractionFlooded', 'MaxExtent']
    df['Year'] = year
    dfs.append(df)
dfs = pd.concat(dfs, ignore_index=True)
dfs['PostTreatment'] = dfs.Year >= 2011
dfs.EVI = np.interp(dfs.EVI, (-2000, 10000), (0.0, 1.0))
rices = []
for year in range(2001, 2022):
    print(year)
    rice = ee.Image("projects/ee-gates/assets/Dataset/Raster/Inferred/RFRiceClassified/"+str(year))
    
    reducedRegions = rice.reduceRegions(districts, ee.Reducer.mean(), 250, 'EPSG:4326')
    
    data = reducedRegions.map(lambda feature: feature.setGeometry(None)).select(['NAME_2','mean']).getInfo()
    dataList = [item['properties'] for item in data['features']]
    df = pd.DataFrame(dataList)
    df.columns = ['District', 'Rice']
    df['Year'] = year
    rices.append(df)
rices = pd.concat(rices, ignore_index=True)
dfs = dfs.merge(rices, on=['District','Year'])
dfs = dfs[['District', 'Year', 'Rice', 'EVI', 'FractionFlooded', 'MaxExtent', 'Duration', 'PostTreatment']]

In [95]:
dfs.to_json('data.json')

In [96]:
dfs

,EVI,District,FractionFlooded,MaxExtent,Year,Duration,PostTreatment,Rice
0,0.446823,Barguna,0.000071,0.000000,2001,0.000071,False,0.664750
1,0.448199,Barishal,0.003017,0.000000,2001,0.003855,False,0.465179
2,0.416707,Bhola,0.000076,0.000040,2001,0.000076,False,0.392893
3,0.471272,Jhalakathi,0.000000,0.000000,2001,0.000000,False,0.564581
4,0.423399,Patuakhali,0.000071,0.000000,2001,0.000071,False,0.659248
...,...,...,...,...,...,...,...,...
1339,0.380135,Sirajganj,0.251555,0.219896,2021,1.067323,True,0.406424
1340,0.423495,Habiganj,0.025941,0.013733,2021,0.239869,True,0.175233
1341,0.484396,Moulvibazar,0.008450,0.005347,2021,0.146313,True,0.197079
1342,0.287785,Sunamganj,0.231194,0.168491,2021,7.200530,True,0.048852


In [64]:
dfs.to_csv('data.csv', index=False)